In [2]:
from gurobipy import GRB
import gurobipy as gb

# Create the optimization model
model = gb.Model("Sunnyshore Bay Project")

In [3]:
# Constants
revenues = [180000, 260000, 420000, 580000]
expenses = [300000, 400000, 350000, 200000]
interest_rates = {1: 0.0175, 2: 0.0225, 3: 0.0275}
borrowing_limits = {0: 250000, 1: 150000, 2: 350000}
min_cash_balances = [25000, 20000, 35000, 18000]

In [4]:
# Create the optimization model
model = gb.Model("Sunnyshore Bay")

# Add decision variables
Borrow = model.addVars(4, 3, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")  
CashBalance = model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="CashBalance")


In [5]:
# Add constraints
# Borrowing constraints
for i in range(4):
    model.addConstr(sum(Borrow[i, j] for j in range(3)) <= borrowing_limits.get(i, float('inf')), f"Borrowing_Limit_Month_{i+1}")

# Cash balance constraints and repayment
for i in range(4):
    income = revenues[i] if i < 4 else 0
    expense = expenses[i] if i < 4 else 0
    model.addConstr(
        CashBalance[i] == (CashBalance[i-1] if i > 0 else 0) + income - expense + sum(Borrow[i, j] for j in range(3)) - sum(Borrow[i-j-1, j] * (1 + interest_rates[j+1]) for j in range(min(i, 3))),
        f"Cash_Balance_Month_{i+1}"
    )
    model.addConstr(CashBalance[i] >= min_cash_balances[i], f"Min_Cash_Balance_Month_{i+1}")

# Constraint for cash balance at the end of July
model.addConstr(CashBalance[2] >= 0.65 * (CashBalance[0] + CashBalance[1]), "Cash_Balance_July")


<gurobi.Constr *Awaiting Model Update*>

In [6]:
# The objective function is to minimize the total repayment amount
model.setObjective(sum(Borrow[i, j] * (1 + interest_rates[j+1]) for i in range(4) for j in range(min(3, 4-i))), GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Output the solution
print("Total amount repaid to the bank: ", model.objVal)
for v in model.getVars():
    print(f"{v.varName}: {v.x}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.4.0 22E261)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 16 columns and 44 nonzeros
Model fingerprint: 0xe944467e
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 7 rows and 7 columns
Presolve time: 0.02s
Presolved: 6 rows, 9 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.000000e+04   0.000000e+00      0s
       3    2.8562500e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.856250000e+05
Total amount repaid to the bank:  2856

In [7]:
# Output the solution
print("Total amount repaid to the bank: ", model.objVal)
for i in range(4):
    print(f"Cash Balance at the end of Month {i+1}: {CashBalance[i].x}")
    for j in range(3):
        if i+j < 4:  # Ensuring we don't access months beyond August
            print(f"Amount borrowed in Month {i+1} with a term of {j+1} month(s): {Borrow[i, j].x}")


Total amount repaid to the bank:  285625.0
Cash Balance at the end of Month 1: 25000.0
Amount borrowed in Month 1 with a term of 1 month(s): 0.0
Amount borrowed in Month 1 with a term of 2 month(s): 145000.0
Amount borrowed in Month 1 with a term of 3 month(s): 0.0
Cash Balance at the end of Month 2: 20000.0
Amount borrowed in Month 2 with a term of 1 month(s): 135000.0
Amount borrowed in Month 2 with a term of 2 month(s): 0.0
Amount borrowed in Month 2 with a term of 3 month(s): 0.0
Cash Balance at the end of Month 3: 35000.0
Amount borrowed in Month 3 with a term of 1 month(s): 0.0
Amount borrowed in Month 3 with a term of 2 month(s): 0.0
Cash Balance at the end of Month 4: 415000.0
Amount borrowed in Month 4 with a term of 1 month(s): 0.0


# (a) How many different investments can be made over the 4-month period?

This might be a combination problem:

May: 4 options (no borrowing, 1-month, 2-months, 3-months)

June: 3 options (no borrowing, 1-month, 2-months)

July: 2 options (no borrowing, 1-month)

August: 1 option (no borrowing)

total combination = 4x3x2x1 = 24 investments

# (b) Write down the cash balance constraint for money on-hand at the end of June.

In [10]:
# Cash balance at the end of June
model.addConstr(
    CashBalance[1] == CashBalance[0] + revenues[1] - expenses[1] + 
    sum(Borrow[1, j] for j in range(3)) - 
    sum(Borrow[0, j] * (1 + interest_rates[j+1]) for j in range(1)),
    "Cash_Balance_Month_2"
)
model.addConstr(CashBalance[1] >= min_cash_balances[1], "Min_Cash_Balance_Month_2")
# Solve the model
model.optimize()

# Output the cash balance at the end of June
print("Cash Balance at the end of June: ", CashBalance[1].x)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.4.0 22E261)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 16 columns and 58 nonzeros
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8562500e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.856250000e+05
Cash Balance at the end of June:  20000.0


# (c) Write down the linear ratio constraint associated with the cash balance at the end of July

In [11]:
model.addConstr(CashBalance[2] >= 0.65 * (CashBalance[0] + CashBalance[1]), "Cash_Balance_July")
# Output the cash balance at the end of July and the required minimum (65% of the combined cash balances of May and June)
print("Cash Balance at the end of July: ", CashBalance[2].x)
print("Required Minimum Cash Balance at the end of July (65% of May and June): ", 0.65 * (CashBalance[0].x + CashBalance[1].x))


Cash Balance at the end of July:  35000.0
Required Minimum Cash Balance at the end of July (65% of May and June):  29250.0


# (d) What is the total amount that Sunnyshore Bay has to repay to the bank over the entire season?

In [12]:
# After solving the model
print("Total amount repaid to the bank over the entire season: ", model.objVal)


Total amount repaid to the bank over the entire season:  285625.0


# (e) How much money does Sunnyshore Bay withdraw in May from all loans?

In [13]:
May_borrowed_total = sum(Borrow[0, j].x for j in range(3))
print("Total amount borrowed in May: ", May_borrowed_total)


Total amount borrowed in May:  145000.0


# (f) What is the cash balance at the end of August?

In [16]:
print("Cash Balance at the end of August: ", CashBalance[3].x)

Cash Balance at the end of August:  415000.0


# (g) Due to potential unexpected repairs, one of the managers has suggested increasing the minimum cash balance for June to $27,500. How much will now have to be repaid if this change is approved?

In [17]:
# Update the minimum cash balance for June
min_cash_balances[1] = 27500

# Update the constraint for June's cash balance
model.remove(model.getConstrByName("Min_Cash_Balance_Month_2"))
model.addConstr(CashBalance[1] >= min_cash_balances[1], "Min_Cash_Balance_Month_2")

# Re-optimize the model
model.optimize()

# Output the new total amount that needs to be repaid
print("New total amount repaid to the bank after increasing June's minimum cash balance: ", model.objVal)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.4.0 22E261)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 16 columns and 61 nonzeros
Model fingerprint: 0x7bd652dd
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 12 rows and 7 columns
Presolve time: 0.02s
Presolved: 6 rows, 9 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.187500e+04   0.000000e+00      0s
       3    2.9325625e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.932562500e+05
New total amount repaid to the bank a

# (h) Formulate and solve the dual linear program demonstrating that the model you create is, indeed, the correct dual problem of the primal formulation.

# (i) Which formulation, the primal or the dual model, do you think is easier to solve?